<img height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px">

<h1>2019 Deep Learning _ HW5_Example</h1>

Sentiment Analysis using RNN (LSTM Example using Keras)

- Myeong-Ha Hwang

In [0]:
import numpy as np 
import pandas as pd 
import nltk
#nltk.download('all') #You only use this line for nltk sub package at first time 
import numpy as np

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from keras.utils import to_categorical
import random
from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
lemmatizer = WordNetLemmatizer()

#set random seed for the session and also for tensorflow that runs in background for keras
set_random_seed(123)
random.seed(123)

In [0]:
# Data upload from windows
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file"{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving test.tsv to test.tsv
Saving train.tsv to train.tsv
User uploaded file"test.tsv" with length 3367149 bytes
User uploaded file"train.tsv" with length 8481022 bytes


In [0]:
# Data loading from colab
import pandas as pd
import io
train = pd.read_csv(io.StringIO(uploaded['train.tsv'].decode('utf-8')), sep='\t')
test = pd.read_csv(io.StringIO(uploaded['test.tsv'].decode('utf-8')), sep='\t')
train.head()
#print(train.head())
#print(test.head())
#print(train.shape)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
#Preprocessing Reference: https://www.kaggle.com/chiranjeevbit/movie-review-prediction/notebook

#Method for cleaning the reviews, tokenize and lemmatize them.
def clean_sentences(df):
    reviews = []

    for sent in tqdm(df['Phrase']):
        
        #remove html content
        review_text = BeautifulSoup(sent).get_text()
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        #tokenize the sentences
        words = word_tokenize(review_text.lower())
    
        #lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)

#cleaned reviews for both train and test set retrieved
train_sentences = clean_sentences(train)
test_sentences = clean_sentences(test)
print(len(train_sentences))
print(len(test_sentences))

100%|██████████| 66292/66292 [00:36<00:00, 1840.29it/s]

156060
66292


In [0]:
#Collect the dependent values and convert to one-hot encoded output using to_categorical
target = train.Sentiment.values
y_target = to_categorical(target)
num_classes = y_target.shape[1]

#Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_sentences, y_target, test_size=0.2, stratify=y_target)

#Getting the no of unique words and max length of a review available in the list of cleaned reviews
unique_words = set()
len_max = 0

for sent in tqdm(X_train):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)
        
#length of the list of unique_words gives the no of unique words
print(len(list(unique_words)))
print(len_max)

100%|██████████| 124848/124848 [00:00<00:00, 531984.00it/s]

13735
48


In [0]:
#Actual tokenizer of keras and convert to sequences

tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(test_sentences)

#padding done to equalize the lengths of all input reviews. LSTM networks needs all inputs to be same length.
#Therefore reviews lesser than max length will be made equal using extra zeros at end. This is padding.
X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
X_test = sequence.pad_sequences(X_test, maxlen=len_max)
print(X_train.shape,X_val.shape,X_test.shape)

(124848, 48) (31212, 48) (66292, 48)


In [0]:
#Early stopping to prevent overfitting
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 2)
callback = [early_stopping]

#Model using Keras LSTM
model=Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 300)           4120500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 48, 128)           219648    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)      

In [0]:
#Fit the model

#This is done for learning purpose only. One can play around with different hyper parameters combinations
#and try increase the accuracy even more. For example, a different learning rate, an extra dense layer 
# before output layer, etc. Cross validation could be used to evaluate the model and grid search 
# further to find unique combination of parameters that give maximum accuracy. This model has a validation
#accuracy of around 66.5%

history=model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=6, batch_size=256, verbose=1, callbacks=callback)

Instructions for updating:
Use tf.cast instead.
Train on 124848 samples, validate on 31212 samples
Epoch 1/6
124848/124848 [==============================] - 133s 1ms/step - loss: 1.0028 - acc: 0.5964 - val_loss: 0.8480 - val_acc: 0.6539
Epoch 2/6
124848/124848 [==============================] - 129s 1ms/step - loss: 0.8078 - acc: 0.6689 - val_loss: 0.8022 - val_acc: 0.6704
Epoch 3/6
124848/124848 [==============================] - 128s 1ms/step - loss: 0.7374 - acc: 0.6935 - val_loss: 0.8101 - val_acc: 0.6733
Epoch 4/6
124848/124848 [==============================] - 129s 1ms/step - loss: 0.6987 - acc: 0.7066 - val_loss: 0.8396 - val_acc: 0.6760
Epoch 5/6
124848/124848 [==============================] - 129s 1ms/step - loss: 0.6719 - acc: 0.7185 - val_loss: 0.8680 - val_acc: 0.6679
Epoch 6/6
124848/124848 [==============================] - 130s 1ms/step - loss: 0.6524 - acc: 0.7276 - val_loss: 0.8751 - val_acc: 0.6693


In [1]:
#Plotting
import matplotlib.pyplot as plt

# Create count of the number of epochs
epoch_count = range(1, len(history.history['loss']) + 1)

# Visualize learning curve. Here learning curve is not ideal. It should be much smoother as it decreases.
#As mentioned before, altering different hyper parameters especially learning rate can have a positive impact
#on accuracy and learning curve.
plt.plot(epoch_count, history.history['loss'], 'g--')
plt.plot(epoch_count, history.history['val_loss'], 'r-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

NameError: ignored

In [0]:
#make the predictions with trained model and submit the predictions.
y_pred=model.predict_classes(X_test)


In [0]:
uploaded = files.upload()


Saving sampleSubmission.csv to sampleSubmission.csv


In [0]:
for fn in uploaded.keys():
    print('User uploaded file"{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    
#sub_file = pd.read_csv('../input/sampleSubmission.csv',sep=',')
sub_file = pd.read_csv(io.StringIO(uploaded['sampleSubmission.csv'].decode('utf-8')))
sub_file.Sentiment=y_pred
sub_file.to_csv('Submission.csv',index=False)

User uploaded file"sampleSubmission.csv" with length 596647 bytes


In [0]:
#Submission data download
files.download('Submission.csv')